# Chapter 4: CAN-Bus

As you can imagine, cars don't use ROS to publish and subscribe to data from the car.<br>
The use of a **bus standard named CAN** is widespread, created by the people at Bosch.<br>
CAN-Bus is one of the five protocols used in the on-board diagnostics (OBD)-II vehicle diagnostics standard.<br>
Its main property is the ability to multiplex electrical wiring, which comes in handy for avoiding the need for hundreds of different cables to send and receive information from all the electrical devices in a system.

If you are familiar with CAN-Bus, you can skip to the first exercise, U4.1.

## What Do CAN-Bus messages look like?

There is a huge variety of CAN-Bus flavors, although the core is more or less the same.<br>
Here we will concentrate on the essentials to understand what is going on and be able to send messages through CAN-Bus.

So, a CAN-Bus message has three main parts:

* Identifier <span style="color: green">(green)</span>: It's 11 bits long.
* Data length code (DLC) <span style="color: yellow">(yellow)</span>: It's 4 bits long.
* Data field <span style="color: red">(red)</span>: It's 0–64 bits (0-8 bytes) long.

<img src="img/CAN-Bus_message.png"/>

It has more elements, but as fas as we are concerned, these are the essential parts.<br>
So, each sensor will be watching for its own identifier, which is the only information that matters to it.<br>
It will also publish the data this sensor generates with the same identifier.<br>
There are basicaly two "cables." They are normally referred to as **tx** and **rx**. You can think of them as tx being the CAN-Bus output, and rx being the CAN-Bus input; although, as we said earlier, there is much more to it than that.

## How do I use CAN-Bus in the simulation?

The simulation you are using is designed so that it works as a real car would.<br>
This means that all of the information sent and received from the physical car comes from the CAN-Bus.<br>
Let's have a look at the node configuration:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in Webshell #1</p>
</th>
</tr>
</table>

In [ ]:
rqt_graph

<table style="float:left;">
<tr>
<th>
Go to the graphical interface window (hit the icon with a screen in the IDE) 
<br><br>
<div class='white_bg'><img src="img/font-awesome_desktop.png"/><br></div>
</th>
</tr>
</table>

You will get something similar to the following:

[Autonomous Vehicle ROS Node and Topics ALL](extra_files/autonomous_U4_rosgraph_original.png)<br>

This graph is quite complex. That's because you are visualizing the entire simulated system.<br>
As far as you are concerned, you only need to have the following structure in mind:

[Autonomous Vehicle ROS Node and Topics Real](extra_files/autonomous_U4_rosgraph_simple_real.png)<br>

This is how it would be if you used a real car.<br>
You would only have to publish into the **/can_bus_dbw/can_tx** to make the car move, the lights activate, and so on. Anytime you wanted to send a message, it would have to be published into this topic. Then, a node would just read from that topic and connect to the CAN-Bus driver device through a USB.
It is the same process to read data. You would receive it from **/can_bus_dbw/can_rx**.

But, as you can see, there is already a node reading from those topics. It's the **/vehicle/dbw_node**. This node is the one in charge of reading the CAN-Bus encoded style topic **/can_bus_dbw/can_rx** and publishing into more user-friendly topics. These user-friendly topics are, for instance, **/vehicle/steering**, **/vehicle/gps/fix**, and **/vehicle/gps/vel**.

The same thing happens when sending information. It reads from standard ROS topics, like **/vehicle/brake_cmd** or **/vehicle/throttle_cmd**, and encodes it into the CAN-Bus format to publish in **/can_bus_dbw/can_tx**.

But, wait! We are not using the real car!

But, as you can see in the following graph, this is how your program will publish into topics that the **/vehicle/dbw_node** will read. And then, publishing into the CAN-Bus topics will be EXACTLY the same as with a real car.

[Autonomous Vehicle ROS Node and Topics Simulation](extra_files/autonomous_U4_rosgraph_simple.png)<br>

The only difference is that instead of using a real CAN-Bus driver device, the system interfaces with a simulated one. But because you are using the same exact CAN-Bus topics (**/can_bus_dbw/can_tx and /can_bus_dbw/can_rx**), there is no difference between working with a REAL or SIMULATED car for you.

### That's the power of ROS.

<p style="background:#EE9023;color:white;">Exercise U4.1</p>

Create a ROS node that moves the car by publishing into the appropriate topics that the dbw_node is subscribed to.<br>
Then, the dbw_node will publish all that information into the Can-Bus output topic **/can_bus_dbw/can_tx**.<br>
Which topics are they? Well, they are the topics that register real information on how a car is controlled, and not through something artificial, like a Twist message.<br><br>
These topics are:<br><br>
**/vehicle/brake_cmd, BrakeCmd from dbw_mkz_msgs package**<br>
**/vehicle/throttle_cmd, ThrottleCmd from dbw_mkz_msgs package**<br>
**/vehicle/steering_cmd, SteeringCmd from dbw_mkz_msgs package**<br>
**/vehicle/gear_cmd, GearCmd from dbw_mkz_msgs package**<br>
<br>
As you can see, you move the car by steering the wheel, and using the brake, throttle, pedals, and gear selection. Just like a normal car.

There are some details that you have to keep in mind to make this work:
<ul>
<li>Before publishing into the aforementioned topics, dbw_node needs to receive a message in the <b>/vehicle/enable</b> topic. This avoids accidental publications.</li>
<li>Make sure that the <b>/vehicle/enable</b> has been published. Because ROS topics do not always work, especially in the first calls, you will have to make sure the topic was published before doing anything else.</li>
<li>
The values in the topic messages are critical in some cases because they might modify the value sent, or not send it at all.<br>
These cases are:<br>
<b>brake_command_object.pedal_cmd_type = 1</b>, Recommended CMD_PEDAL=1 to be able to send unitless values. See BrakeCmd.msg for more options.<br>
<b>throttle_command_object.pedal_cmd_type = 1</b>, Recommended CMD_PEDAL=1 to be able to send unitless values. See ThrottleCmd.msg for more options.<br>
</li>
<li>
The Gear value is also quite arbitrary. Therefore, have a look at the Gear.msg to see the options available. For example, setting the Direct Gear (Forwards) is DRIVE=4, integer value 4.
</li>
</ul>



<p style="background:#EE9023;color:white;">END Exercise U4.1</p>

<p style="background:#AE0202;color:white;">Expected Result for Exercise U4.1</p>

You should be able to move the car in any direction.<br>
Check to make sure that when you press the brake, it goes slower; and when it exceeds certain values, the car stops.<br>
Make sure that you can reverse the car, or that if you don't put D or R, the car doesn't move.

<p style="background:green;color:white;">Solution Exercise U4.1</p>

Please try to do it by yourself, unless you get stuck or need some inspiration. You will learn much more if you fight for each exercise.

<img src="img/robotignite_logo_text.png"/>

In [ ]:
#!/usr/bin/env python
# license removed for brevity
import rospy
import random
from dbw_mkz_msgs.msg import BrakeCmd
from dbw_mkz_msgs.msg import ThrottleCmd
from dbw_mkz_msgs.msg import SteeringCmd
from dbw_mkz_msgs.msg import GearCmd, Gear
from dataspeed_can_msgs.msg import CanMessage
from std_msgs.msg import Empty

class CarControl(object):
    def __init__(self):
        self.pub_brake_cmd = rospy.Publisher('/vehicle/brake_cmd', BrakeCmd, queue_size=1)
        self.pub_throttle_cmd = rospy.Publisher('/vehicle/throttle_cmd', ThrottleCmd, queue_size=1)
        self.pub_steering_cmd = rospy.Publisher('/vehicle/steering_cmd', SteeringCmd, queue_size=1)
        self.pub_gear_cmd = rospy.Publisher('/vehicle/gear_cmd', GearCmd, queue_size=1)
        
        # We create this pub sub to check that the enable is sent, otherwise no publishing will be taken by dbw
        self._enable_dbw = False
        self.pub_enable = rospy.Publisher('/vehicle/enable', Empty, queue_size=1)
        sub = rospy.Subscriber("/vehicle/enable", Empty, self.enable_callback)
        self.enable_dbw_inputs_loop()
        
        # We wait for the Gazebo system to be available before publishing anything
        rospy.wait_for_service('/gazebo/set_physics_properties')
        rospy.loginfo("Gazebo Ready...")

    
    def enable_callback(self,data):
        self._enable_dbw = True

    def enable_dbw_inputs_loop(self):
        """
        We publish enable to enable dbw to allow commands from outside
        rostopic pub /vehicle/enable std_msgs/Empty "{}"
        """
        rate = rospy.Rate(1)
        while not self._enable_dbw:
            rospy.loginfo("Try to enable /vehicle/enable, STATUS =  "+str(self._enable_dbw))
            self.enable_dbw_inputs()
            rate.sleep()
        rospy.loginfo("dbw inputs ENABLED...")

    def enable_dbw_inputs(self):
        
        empty_message = Empty()
        rospy.loginfo("Publish Enable Command in /vehicle/enable ")
        self.pub_enable.publish(empty_message)

    def cmd_publisher(self):
        """
        This makes the simulated function of all the basic controles of a car being used:
        brake, throttle, gear and steering_wheel.
        :return:
        """
        
        rate = rospy.Rate(5)
        speed = 1.0
        while not rospy.is_shutdown():
            rospy.loginfo("########### Starting CarControl Cmd Publish Round... ###########")
            self.throttle_cmd_publisher(throttle_value=speed)
            self.brake_cmd_publisher(brake_value=0.0)
            self.steering_cmd_publisher(angle_rad=0.0)
            self.gear_cmd_publisher(gear_value=4)
            rate.sleep()
            
    
    def throttle_cmd_publisher(self, throttle_value=0.0):
        """
        ID = 98 in BUSCAN
        # Throttle pedal
        # Options defined below
        float32 pedal_cmd
        uint8 pedal_cmd_type
        
        # Enable
        bool enable
        
        # Ignore driver overrides
        bool ignore
        
        # Watchdog counter (optional)
        uint8 count
        
        uint8 CMD_NONE=0
        uint8 CMD_PEDAL=1   # Unitless, range 0.15 to 0.80
        uint8 CMD_PERCENT=2 # Percent of maximum throttle, range 0 to 1
        """
        #msg_str = "Publishing Throttle message in /vehicle/throttle_cmd %s" % rospy.get_time()
        #rospy.loginfo(msg_str)
        throttle_command_object = ThrottleCmd()
        #throttle_command_object.pedal_cmd = random.uniform(0.15, 0.8)
        throttle_command_object.pedal_cmd = throttle_value
        throttle_command_object.pedal_cmd_type = 1
        throttle_command_object.enable = False
        throttle_command_object.ignore = False
        throttle_command_object.count = 0
        rospy.loginfo("Throttle Publish ::>"+str(throttle_command_object.pedal_cmd))
        self.pub_throttle_cmd.publish(throttle_command_object)
    
    def brake_cmd_publisher(self, brake_value=0.0):
        """
        ID = 96 in BusCan
        # Brake pedal
        # Options defined below
        float32 pedal_cmd
        uint8 pedal_cmd_type
        
        # Brake On Off (BOO), brake lights
        bool boo_cmd
        
        # Enable
        bool enable
        
        # Ignore driver overrides
        bool ignore
        
        # Watchdog counter (optional)
        uint8 count
        
        uint8 CMD_NONE=0
        uint8 CMD_PEDAL=1   # Unitless, range 0.15 to 0.50
        uint8 CMD_PERCENT=2 # Percent of maximum torque, range 0 to 1
        uint8 CMD_TORQUE=3  # Nm, range 0 to 3250
        
        float32 TORQUE_BOO=520  # Nm, brake lights threshold
        float32 TORQUE_MAX=3412 # Nm, maximum torque
        """
        #msg_str = "Publishing Brake message in /vehicle/brake_cmd %s" % rospy.get_time()
        #rospy.loginfo(msg_str)
        brake_command_object = BrakeCmd()
        #brake_command_object.pedal_cmd = random.uniform(0.15, 0.5)
        brake_command_object.pedal_cmd = brake_value
        brake_command_object.pedal_cmd_type = 1
        brake_command_object.boo_cmd = False
        brake_command_object.enable = False
        brake_command_object.ignore = False
        brake_command_object.count = 0
        rospy.loginfo("Brake Publish ::>"+str(brake_command_object.pedal_cmd))
        self.pub_brake_cmd.publish(brake_command_object)
    
    def steering_cmd_publisher(self, angle_rad=0.0):
        """
        ID = 100 in BusCan
        # Steering Wheel
        float32 steering_wheel_angle_cmd        # rad, range -8.2 to 8.2
        float32 steering_wheel_angle_velocity   # rad/s, range 0 to 8.7, 0 = maximum
        
        # Enable
        bool enable
        
        # Ignore driver overrides
        bool ignore
        
        # Disable the driver override audible warning
        bool quiet
        
        # Watchdog counter (optional)
        uint8 count
        """
        #msg_str = "Publishing Steering message in /vehicle/steering_cmd %s" % rospy.get_time()
        #rospy.loginfo(msg_str)
        steering_command_object = SteeringCmd()
        #steering_command_object.steering_wheel_angle_cmd = random.uniform(-8.2, 8.2)
        steering_command_object.steering_wheel_angle_cmd = angle_rad
        steering_command_object.steering_wheel_angle_velocity = random.uniform(0.0, 8.7)
        #steering_command_object.steering_wheel_angle_velocity = 0.8
        
        steering_command_object.enable = False
        steering_command_object.ignore = False
        steering_command_object.quiet = False
        steering_command_object.count = 0
        rospy.loginfo("Steering Publish ::>"+str(steering_command_object.steering_wheel_angle_cmd))
        self.pub_steering_cmd.publish(steering_command_object)
        
    def gear_cmd_publisher(self,gear_value=0):
        """
        ID = 102 in BusCan
        # Gear command enumeration
        Gear cmd
        >>>
        
        uint8 gear
        
        uint8 NONE=0
        uint8 PARK=1
        uint8 REVERSE=2
        uint8 NEUTRAL=3
        uint8 DRIVE=4
        uint8 LOW=5
        >>>
        
        gear is an uint8, but in the MsgGearCmd in dispatch.h only gets 3 Bits, which make up to
        8 (0-7) possible gears, which could be: 1,2,3,4,5,R,P,N.
        But in th emessage deffinition the only options given are None, Park, Reverse, Neutral, Drive and Low.
        so 6 options (0-5)
        """
        #msg_str = "Publishing Gear message in /vehicle/gear_cmd %s" % rospy.get_time()
        #rospy.loginfo(msg_str)
        gear_command_object = GearCmd()
        gear_object = Gear()
        #gear_object.gear = random.randint(0, 5)
        gear_object.gear = gear_value
        gear_command_object.cmd = gear_object
        rospy.loginfo("Gear Publish ::>"+str(gear_object.gear))
        self.pub_gear_cmd.publish(gear_command_object)
    



if __name__ == '__main__':
    rospy.init_node('sterring_throttle_brake_gear_publisher_node', anonymous=True)
    car_control_object = CarControl()
    try:
        car_control_object.cmd_publisher()
    except rospy.ROSInterruptException:
        print "Exception occured"
        pass

<p style="background:green;color:white;">Solution Exercise U4.1</p>

Okay, great! Now you can move the SIMULATED car or the REAL car with the node you've just created.

## Congratulations!!

If you are interested in learning more about CAN-Bus, in ROS or in general, stay a bit longer.<br>
Just bear in mind that you <b>will have to know C++ to be able to understand some parts.<b>

<p style="background:#EE9023;color:white;">Exercise EXTRA U4.2</p>

The objective of this exercise is to show you the basics of how the CAN-Bus encoding works, and how this particular system does it.<br>
The exercise is the following:<br>
Write a node that is subscribed to both CAN topics and decodes the CAN encoded data.<br>
Things to keep in mind:<br>
<ul>
<li>First, just subscribe to the topics <b>/can_bus_dbw/can_rx</b> and <b>/can_bus_dbw/can_tx</b>. See the data that is sent? Try to identify the CAN-Bus structure you learned in the intro of this unit. Can you get the ID? The data? The data length code?</li>
<li>
Indentify which ID corresponds to which kind of information. Is it throttle information? Brake?<br>
To know this, you will have to access the <b>dbw_mkz_can package</b>, the dispatch.h file inside the included folder.<br>
Here you have the extract of the data you need. Keep in mind that the numbers are in HEX format, so change them to DECIMAL format first.
</li>
<li style="color:red">Please be aware that the brake, throttle, steering. and gear information are published in the /can_bus_dbw/can_tx (output); while the GPS data comes from the /can_bus_dbw/can_rx (input)</li>
</ul>

<ul>
<li>Now, for the hardcore people, decode the data from each transmission.<br>
For this, you will have to dig deep into the code.<br>
You need to know how each topic message is processed and converted to CAN.<br>
This way, you will be able to reconvert the CAN data to standard data.<br>
Here you have a snippet on how the GearCmd is processed and published in the can_tx, and also how a more complex one publishes the SteeringCmd. If you want to see all the messages, you will have to take a look at the <b>DbwNode.cpp</b> in the <b>dbw_mkz_can</b> package.
Try these two first and then dig into the throttle and brake.
</li>
<li>All the cpp and h files have been copied to a folder for your easy access. This folder is in the package <b>dbw_mkz_can</b> and it's called <b>extra_docs</b>. Here you can find <b>DbwNode.cpp</b>, <b>DbwNode.h</b>, and <b>dispatch.h</b>.</li>
</ul>

<ul>
<li>Now to read from the <b>/can_bus_dbw/can_rx</b>, the most difficult of all: The GPS.<br>
GPS is really challenging because it's made up of three different CAN-Bus messages that have to be grouped by time and then generate the data. This is the snippet where the GPS data is encoded into the three different CAN-Bus messages:</li>
</ul>

<p style="background:#AE0202;color:white;">Expected Result for Exercise U4.2</p>

The sent data must coincide with the data read from the CAN-Bus topics.<br>
In the case of the <b>/can_bus_dbw/can_rx</b> GPS data, it has to coincide with the data published in the /fix, and /fix_velocity published by the simulation.

<p style="background:#EE9023;color:white;">END EXTRA Exercise U4.2</p>

Further References:<br>
https://www.kvaser.com/can-protocol-tutorial/<br>
http://www.socialledge.com/sjsu/index.php?title=CAN_BUS_Tutorial<br>
https://es.wikipedia.org/wiki/Bus_CAN<br>
http://www.canbus.us/<br>
<br>
Books:<br>
https://www.amazon.es/Comprehensible-Guide-Controller-Area-Network/dp/0976511606<br>
http://www.canbus.us/index.php/links/can-books<br>

Original Protocol:
http://www.bosch-semiconductors.de/media/ubk_semiconductors/pdf_1/canliteratur/can2spec.pdf

CAN-Bus Hardware:
https://bitbucket.org/DataspeedInc/dbw_mkz_ros/downloads/ADAS_Kit_20161206.zip

Image Source Udacity Car Real:
http://www.creativeshot.com/corporate/commercial/udacity-self-driving-nanodegree-program/